Course work

Author: sushmaakoju

## Baseline LLM4Judge challenge

### Configuring the baseline

Changes required for the quick-start:

- bitsandbytes==0.43.1
- add quantization
- use dataset: query, passage pairs of texts
- add quantization for low cpu memory usage for transformers pipeline
- add BitsAndBytesConfig quantization for low cpu memory usage for transformers AutoModel (for CUDA memory)

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# !sudo apt-get install -y nvidia-smi

In [3]:
!pip install jsonlines
!pip install transformers

!pip install bitsandbytes==0.43.1
!python3 -m bitsandbytes

!pip install accelerate
!pip install torch torchvision
!pip install datasets
!pip install --upgrade huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [4]:
import torch

import bitsandbytes as bnb

p = torch.nn.Parameter(torch.rand(10, 10).cuda())
a = torch.rand(10, 10).cuda()

p1 = p.data.sum().item()

adam = bnb.optim.Adam([p])

out = a * p
loss = out.sum()
loss.backward()
adam.step()

p2 = p.data.sum().item()

assert p1 != p2
print("SUCCESS!")
print("Installation was successful!")

SUCCESS!
Installation was successful!


In [5]:
!nvidia-smi

Thu Jun  6 09:04:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              29W /  70W |    121MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
!git clone https://github.com/llm4eval/LLMJudge.git

Cloning into 'LLMJudge'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 7), reused 14 (delta 2), pack-reused 0
Receiving objects: 100% (34/34), 2.33 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [7]:
# !huggingface-cli login --token $HF_TOKEN

In [8]:
!huggingface-cli logout
!huggingface-cli login
#!huggingface-cli whoami
# huggingface-cli download meta-llama/Meta-Llama-3-8B-Instruct --include "original/*" --local-dir meta-llama/Meta-Llama-3-8B-Instruct

Not logged in!

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [9]:
import pandas as pd
import jsonlines
from tqdm import tqdm

import transformers
from transformers.utils import is_accelerate_available, is_bitsandbytes_available, is_torch_available
import torch

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, set_seed
import time

from transformers import (AutoTokenizer,AutoModelForCausalLM,
                          TextStreamer,pipeline,BitsAndBytesConfig)

In [10]:
torch.set_default_device("cuda")
torch.cuda.is_available()
model_path = "/content/drive/MyDrive/meta-llama-3-8B-instruct"

In [11]:
docid_to_doc = dict()

with jsonlines.open('/content/LLMJudge/data/llm4eval_document_2024.jsonl', 'r') as document_file:
  for obj in document_file:
    docid_to_doc[obj['docid']] = obj['doc']

In [12]:
query_data = pd.read_csv("/content/LLMJudge/data/llm4eval_query_2024.txt", sep="\t", header=None, names=['qid', 'qtext'])
qid_to_query = dict(zip(query_data.qid, query_data.qtext))

In [13]:
system_message = """You are a search quality rater evaluating the relevance of passages. Given a query and passage, you must provide a score on an integer scale of 0 to 3 with the following meanings:

    3 = Perfectly relevant: The passage is dedicated to the query and contains the exact answer.
    2 = Highly relevant: The passage has some answer for the query, but the answer may be a bit unclear, or hidden amongst extraneous information.
    1 = Related: The passage seems related to the query but does not answer it.
    0 = Irrelevant: The passage has nothing to do with the query

    Assume that you are writing an answer to the query. If the passage seems to be related to the query but does not include any answer to the query, mark it 1. If you would use any of the information contained in the passage in such an asnwer, mark it 2. If the passage is primarily about the query, or contains vital information about the topic, mark it 3. Otherwise, mark it 0."""


In [14]:
def get_prompt(query, passage):
    return f"""Please rate how the given passage is relevant to the query. The output must be only a score that indicate how relevant they are.

    Query: {query}
    Passage: {passage}

    Score:"""

In [15]:
print(torch.__version__)
print(is_torch_available())
torch.cuda.empty_cache()

2.3.0+cu121
True


In [16]:
test_qrel = pd.read_csv("/content/LLMJudge/data/llm4eval_test_qrel_2024.txt", sep=" ", header=None, names=['qid', 'Q0', 'docid'])
test_qrel.head(5)

,qid,Q0,docid
0,q49,0,p3659
1,q49,0,p11027
2,q49,0,p1270
3,q49,0,p786
4,q49,0,p9577


In [17]:
len(test_qrel)

4423

In [18]:
test_query_doc = []
test_query_doc_list = []
for eachline in test_qrel.itertuples(index=True):
  qidx = eachline.qid
  docidx = eachline.docid
  query=qid_to_query[qidx]
  passage=docid_to_doc[docidx]
  prompt = get_prompt(query=qid_to_query[qidx], passage=docid_to_doc[docidx])
  test_query_doc_list.append([qidx, docidx, prompt])
  test_query_doc.append({"qid":qidx, "query":query, "docid":docidx, "passage":passage, "prompt":prompt})

In [19]:
get_prompt(query=qid_to_query[qidx], passage=docid_to_doc[docidx])

'Please rate how the given passage is relevant to the query. The output must be only a score that indicate how relevant they are.\n\n    Query: How does being considerate and friendly affect your relationship with people?\n    Passage: Strengths. The October 3rd meaning shows that you are going to be an expressive and talkative person who is considerate and caring. You are one of the ardent believers of peace and harmony in a relationship; thus, you allow it to reflect in your relationship with people.\n\n    Score:'

In [20]:
test_query_doc_list[:4]

[['q49',
  'p3659',
  'Please rate how the given passage is relevant to the query. The output must be only a score that indicate how relevant they are.\n\n    Query: how does a bounty hunter make money\n    Passage: Other terms for this job are bail enforcement agent, fugitive recovery agent, and bail fugitive investigator. The job description is simple – a bounty hunter captures fugitives from justice in exchange for a monetary reward (the bounty).\n\n    Score:'],
 ['q49',
  'p11027',
  'Please rate how the given passage is relevant to the query. The output must be only a score that indicate how relevant they are.\n\n    Query: how does a bounty hunter make money\n    Passage: If the defendant eludes bail by not showing up for court, it is the bail bondsman who is responsible for paying the remaining 90 percent of the bond to the court. Bounty hunters are most often hired by bail bondsmen to recover a fugitive and return him to the court.\n\n    Score:'],
 ['q49',
  'p1270',
  'Pleas

In [21]:
test_df = pd.DataFrame.from_dict(test_query_doc)
test_df.head()

,qid,query,docid,passage,prompt
0,q49,how does a bounty hunter make money,p3659,Other terms for this job are bail enforcement ...,Please rate how the given passage is relevant ...
1,q49,how does a bounty hunter make money,p11027,If the defendant eludes bail by not showing up...,Please rate how the given passage is relevant ...
2,q49,how does a bounty hunter make money,p1270,Bounty Hunter Salary. Successful bail bond inv...,Please rate how the given passage is relevant ...
3,q49,how does a bounty hunter make money,p786,Bail Enforcement: A Bail Enforcement Agent or ...,Please rate how the given passage is relevant ...
4,q49,how does a bounty hunter make money,p9577,Bounty Hunter: A bounty hunter is an individua...,Please rate how the given passage is relevant ...


In [22]:
test_df.head(1)["prompt"].values

array(['Please rate how the given passage is relevant to the query. The output must be only a score that indicate how relevant they are.\n\n    Query: how does a bounty hunter make money\n    Passage: Other terms for this job are bail enforcement agent, fugitive recovery agent, and bail fugitive investigator. The job description is simple – a bounty hunter captures fugitives from justice in exchange for a monetary reward (the bounty).\n\n    Score:'],
      dtype=object)

In [23]:
!nvidia-smi

Thu Jun  6 09:05:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0              30W /  70W |    121MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Transformers Pipeline

In [24]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

torch.cuda.empty_cache()
time.sleep(60)
# model = transformers.AutoModelForCausalLM.from_pretrained(model_path)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16,
                  #"load_in_4bit": True, --> without quantization
                  "quantization_config": {"load_in_4bit": True},
                  "low_cpu_mem_usage": True,},
    device_map="auto",
    #device="cuda",
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
def get_relevance_score(prompt):
  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": prompt},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      pad_token_id=128009,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  return outputs[0]["generated_text"][len(prompt):]

In [ ]:
with open('llm4eval_test_qrel_results.txt', 'w') as result_file:
  for eachline in tqdm(test_qrel.itertuples(index=True)):
    qidx = eachline.qid
    docidx = eachline.docid
    prompt = get_prompt(query=qid_to_query[qidx], passage=docid_to_doc[docidx])
    pred_score = get_relevance_score(prompt)
    result_file.write(f"{qidx} 0 {docidx} {pred_score}\n")

10it [00:29,  2.92s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
68it [03:16,  2.87s/it]

## Transformers AutoModelForCausalLM

In [53]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #torch_dtype=torch.bfloat16,
    quantization_config = bnb_config,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Test AutoModel with Quantization configuration with an example

In [54]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    pad_token_id=128009,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Arrrr, me hearty! I be Captain Chat, the scurviest chatbot to ever sail the Seven Seas o' the Interwebs! Me and me trusty crew o' code have been programmed to respond to yer queries with the swashbucklin' flair o' a seasoned buccaneer! So hoist the colors, me matey, and let's set sail fer a chat full o' adventure and fun!


### Main Module

In [29]:
len(test_query_doc)

4423

In [35]:
ds = DatasetDict()
ds["test"] = Dataset.from_pandas(test_df)

In [71]:
def get_relevance_score2(prompt):
  messages = [
      {"role": "system", "content": system_message},
      {"role": "user", "content": prompt},
  ]

  prompt = tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
  ).to(model.device)

  terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      pad_token_id=128009,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  response = outputs[0][input_ids.shape[-1]:]

In [ ]:
with open('llm4eval_test_qrel_results_2.txt', 'w') as result_file:
  for eachline in tqdm(test_qrel.itertuples(index=True)):
    qidx = eachline.qid
    docidx = eachline.docid
    prompt = get_prompt(query=qid_to_query[qidx], passage=docid_to_doc[docidx])
    pred_score = get_relevance_score(prompt)
    result_file.write(f"{qidx} 0 {docidx} {pred_score}\n")

In [72]:
with open('llm4eval_test_qrel_results.txt', 'w') as result_file:
  for row in tqdm(test_query_doc_list):
    prompt = row[2]
    pred_score = get_relevance_score2(row[2])
    qidx = row[0]
    docidx = row[1]
    result_file.write(f"{qidx} 0 {docidx} {pred_score}\n")

100%|██████████| 3/3 [00:01<00:00,  1.50it/s]
